### Check inference done on images with their labels

Labels are saved as .txt files

In [ ]:
import os

for root, dirs, files in os.walk("runs/detect"):
    for file in files:
        print(os.path.join(root, file))

### Functions for iou

In [ ]:
# Set image dimensions as specified when training yolo
img_width = 640
img_height = 640

#convert yolo format to (x1, y1, x2, y2) that are not normalized
def yolo_to_xyxy(xc, yc, w, h):
    x1 = (xc - w / 2) * img_width
    y1 = (yc - h / 2) * img_height
    x2 = (xc + w / 2) * img_width
    y2 = (yc + h / 2) * img_height
    return [x1, y1, x2, y2]

#compute IoU between ground truth and predicted bbox
def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA) #intersection of the areas
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1]) #area of boxA
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1]) #area of boxB
    iou = interArea / (boxAArea + boxBArea - interArea) #getting intersection over union
    return iou

### Function to load each label file into a dictionary

In [ ]:
#load predicted label files by yolo into a dictionary
def load_predictions(pred_dir):
    pred_dict = {}
    for file in os.listdir(pred_dir):
        if file.endswith(".txt"):
            file_path = os.path.join(pred_dir, file)
            with open(file_path, "r") as f:
                lines = f.readlines()
            boxes = []
            for line in lines:
                parts = list(map(float, line.strip().split()))
                if len(parts) == 6:
                    parts = parts[:5]  #retreive only class id and coordinates from yolo file
                boxes.append(parts)  # [class_id, xc, yc, w, h]
            pred_dict[file] = boxes
    return pred_dict

### Function to calculate all IoUs in the image

In [ ]:
from glob import glob

def compute_all_ious(gt_dir, pred_dir):
    pred_dict = load_predictions(pred_dir) #path holding predicted bboxes
    results = {}

    for label_file in glob(os.path.join(gt_dir, "*.txt")): 
        filename = os.path.basename(label_file)
        with open(label_file, "r") as f:
            gt_lines = f.readlines()

        gt_boxes = []
        for line in gt_lines:
            parts = list(map(float, line.strip().split()))
            _, xc, yc, w, h = parts
            gt_boxes.append(yolo_to_xyxy(xc, yc, w, h))

        pred_boxes = []
        if filename in pred_dict:
            for pred in pred_dict[filename]:
                _, xc, yc, w, h = pred
                pred_boxes.append(yolo_to_xyxy(xc, yc, w, h))

        image_ious = []
        for gt_box in gt_boxes:
            best_iou = 0
            for pred_box in pred_boxes:
                iou = compute_iou(gt_box, pred_box)
                if iou > best_iou:
                    best_iou = iou  #preserve only the bboxes with best iou
            image_ious.append(best_iou)

        results[filename] = image_ious

    return results

### Retreiving the IoUs

In [ ]:
# Run the function on your data
gt_dir = '../yolov5_split/val/labels'
first_model_labels = 'runs/detect/exp5/labels'  #path holding the labels of test data by first model
second_model_labels = 'runs/detect/exp4/labels'  #path holding the labels of test data by second model
ious1 = compute_all_ious(gt_dir, first_model_labels)
ious2 = compute_all_ious(gt_dir, second_model_labels)

print(ious1.items())

# Print results
print('IoUs for the 1st model are as follows: ')
for img_name, iou_list in ious1.items():
    print(f"{img_name}: IoUs = {iou_list}")

print('IoUs for the 2nd model are as follows: ')
for img_name, iou_list in ious2.items():
    print(f"{img_name}: IoUs = {iou_list}")